In [1]:
import pandas as pd
import numpy as np

In [2]:
raw = pd.read_excel("new_table.xlsx", header = 0, sheet_name="new_table")
raw.head()

,식당(ID),식당명,지점명,지역명,어워드정보설명,(RTI)지수,온라인화진행여부,수용태세지수,인기도,트립어드바이저평점,씨트립평점,네이버평점
0,108660,베누스,NaN,양천구,NaN,0.910544,N,0.202,NaN,NaN,NaN,5.0
1,109087,돌솥바베큐,NaN,양천구,NaN,0.910544,N,0.202,NaN,NaN,NaN,5.0
2,109732,화이팅,NaN,양천구,NaN,3.815154,N,0.217,0.01,NaN,NaN,5.0
3,109976,선가네분식,NaN,양천구,NaN,0.910544,N,0.202,NaN,NaN,NaN,5.0
4,110607,칼포니호프,NaN,양천구,NaN,0.910544,N,0.202,NaN,NaN,NaN,5.0


In [3]:
my_area = raw[raw['지역명'].isin(['종로구','양천구','용산구','서대문구','성북구'])][['식당(ID)','식당명', '지점명','지역명','네이버평점']]

my_area.head(10)

,식당(ID),식당명,지점명,지역명,네이버평점
0,108660,베누스,NaN,양천구,5.0
1,109087,돌솥바베큐,NaN,양천구,5.0
2,109732,화이팅,NaN,양천구,5.0
3,109976,선가네분식,NaN,양천구,5.0
4,110607,칼포니호프,NaN,양천구,5.0
5,111053,광장호프,NaN,양천구,5.0
6,111235,섬마을포차,NaN,양천구,5.0
7,111719,금순네,NaN,양천구,5.0
8,111958,연가포차,NaN,양천구,5.0
9,112054,동호정,NaN,양천구,5.0


In [4]:
import numpy as np

# NaN 값을 처리하기 위해 numpy를 import합니다.

my_area['식당명+지점명'] = np.where(
    my_area['지점명'].notna(),
    my_area['식당명'].astype(str) + " " + my_area['지점명'].astype(str),
    my_area['지역명'] + " " + my_area['식당명'].astype(str)
)

my_area.head(10)


,식당(ID),식당명,지점명,지역명,네이버평점,식당명+지점명
0,108660,베누스,NaN,양천구,5.0,양천구 베누스
1,109087,돌솥바베큐,NaN,양천구,5.0,양천구 돌솥바베큐
2,109732,화이팅,NaN,양천구,5.0,양천구 화이팅
3,109976,선가네분식,NaN,양천구,5.0,양천구 선가네분식
4,110607,칼포니호프,NaN,양천구,5.0,양천구 칼포니호프
5,111053,광장호프,NaN,양천구,5.0,양천구 광장호프
6,111235,섬마을포차,NaN,양천구,5.0,양천구 섬마을포차
7,111719,금순네,NaN,양천구,5.0,양천구 금순네
8,111958,연가포차,NaN,양천구,5.0,양천구 연가포차
9,112054,동호정,NaN,양천구,5.0,양천구 동호정


In [5]:
name_list = my_area['식당명+지점명'].tolist()
name_list = name_list
name_list[:10]

['양천구 베누스',
 '양천구 돌솥바베큐',
 '양천구 화이팅',
 '양천구 선가네분식',
 '양천구 칼포니호프',
 '양천구 광장호프',
 '양천구 섬마을포차',
 '양천구 금순네',
 '양천구 연가포차',
 '양천구 동호정']

In [6]:
import requests

res_info_list = []
no_place = []

for searching in name_list:
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
    headers = {
        "Authorization": "KakaoAK b9f489946ca62947a1d9b5320150e4a7"
    }
    places = requests.get(url, headers=headers).json()['documents']
    
    if len(places) <= 0:
        places = {'address_name': np.nan,
                 'category_group_code': np.nan,
                 'category_group_name': np.nan,
                 'category_name': np.nan,
                 'distance': np.nan,
                 'id': np.nan,
                 'phone': np.nan,
                 'place_name': np.nan,
                 'place_url': np.nan,
                 'road_address_name': np.nan,
                 'x': np.nan,
                 'y': np.nan}
        res_info_list.append(places)
        
    else:
        res_info_list.append(places[0])

In [21]:
len(res_info_list)

5000

In [41]:
res_info_df = pd.DataFrame(res_info_list)
res_info_df['평점'] = raw['네이버평점']
res_info_df.loc[res_info_df['address_name'].isna(), '평점'] = np.nan

In [42]:
res_info_df[:10]

,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y,평점
0,서울 구로구 구로동 476-6,,,"가정,생활 > 미용 > 미용실",,678641450,02-866-3511,베누스헤어샵,http://place.map.kakao.com/678641450,서울 구로구 구로동로47길 11-1,126.881830215046,37.4979737893589,5.0
1,서울 양천구 신월동 110-29,FD6,음식점,"음식점 > 술집 > 호프,요리주점",,16847780,02-2695-6643,돌솥바베큐,http://place.map.kakao.com/16847780,서울 양천구 곰달래로5길 22-1,126.833882712082,37.5316406351089,5.0
2,서울 양천구 목동 601-2,FD6,음식점,음식점 > 한식,,16052371,,화이팅,http://place.map.kakao.com/16052371,서울 양천구 목동중앙북로 33-20,126.86828451767764,37.54864378487966,5.0
3,서울 양천구 신정동 1294,FD6,음식점,음식점 > 분식,,20540817,02-2697-8998,선가네분식,http://place.map.kakao.com/20540817,서울 양천구 신정로14길 1,126.844478845194,37.5088957497724,5.0
4,서울 양천구 신월동 62-3,FD6,음식점,"음식점 > 술집 > 호프,요리주점",,365036181,,칼포니호프,http://place.map.kakao.com/365036181,서울 양천구 월정로 229,126.832899709504,37.538667947389,5.0
5,서울 양천구 신월동 85-6,FD6,음식점,"음식점 > 술집 > 호프,요리주점",,16933981,02-2601-1360,호프광장,http://place.map.kakao.com/16933981,서울 양천구 월정로 215,126.833548267272,37.5375507231889,5.0
6,서울 금천구 독산동 974-25,FD6,음식점,"음식점 > 한식 > 해물,생선",,845885022,02-839-7959,섬마을,http://place.map.kakao.com/845885022,서울 금천구 독산로 324,126.903724199835,37.475607220937,5.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,서울 강서구 마곡동 757,FD6,음식점,음식점 > 한식,,1766770993,02-2604-5292,동호정,http://place.map.kakao.com/1766770993,서울 강서구 마곡중앙로 161-8,126.82726603340956,37.569087404945236,5.0


### 2차 데이터 전처리

In [43]:
# 내 지역구가 아닌 지역구
not_my_area = ['강남구', '도봉구', '중구', '중랑구', '광진구', '마포구',
               '금천구', '서초구', '성동구', '동대문구', '송파구', '동작구', 
               '강서구', '노원구', '강동구', '영등포구', '강북구', '관악구', '은평구', '구로구']

# 내 지역구가 아닌 행의 인덱스를 찾음
rows_to_drop = res_info_df[res_info_df['address_name'].str.contains('|'.join(not_my_area), na=False)].index

# 해당 행을 삭제
res_info_df = res_info_df.drop(rows_to_drop)

# reset_index() 함수를 호출하여 인덱스를 다시 정렬
res_info_df = res_info_df.reset_index(drop=True)

In [44]:
# 카카오API로 겁색하지 못한 음식점 개수
res_info_df.isna().sum()

address_name           850
category_group_code    850
category_group_name    850
category_name          850
distance               850
id                     850
phone                  850
place_name             850
place_url              850
road_address_name      850
x                      850
y                      850
평점                     850
dtype: int64

In [45]:
# 서울 아닌 행, NaN값 삭제
rows_to_drop2 = res_info_df[~res_info_df['address_name'].str.startswith('서울', na=False)].index
res_info_df = res_info_df.drop(rows_to_drop2)
res_info_df = res_info_df.reset_index(drop=True)

In [46]:
res_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3968 entries, 0 to 3967
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   address_name         3968 non-null   object 
 1   category_group_code  3968 non-null   object 
 2   category_group_name  3968 non-null   object 
 3   category_name        3968 non-null   object 
 4   distance             3968 non-null   object 
 5   id                   3968 non-null   object 
 6   phone                3968 non-null   object 
 7   place_name           3968 non-null   object 
 8   place_url            3968 non-null   object 
 9   road_address_name    3968 non-null   object 
 10  x                    3968 non-null   object 
 11  y                    3968 non-null   object 
 12  평점                   3968 non-null   float64
dtypes: float64(1), object(12)
memory usage: 403.1+ KB


In [47]:
# 음식점 아닌 행 삭제
rows_to_drop3 = res_info_df[~res_info_df['category_name'].str.startswith('음식점', na=False)].index
res_info_df = res_info_df.drop(rows_to_drop3)
res_info_df = res_info_df.reset_index(drop=True)

In [48]:
res_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3851 entries, 0 to 3850
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   address_name         3851 non-null   object 
 1   category_group_code  3851 non-null   object 
 2   category_group_name  3851 non-null   object 
 3   category_name        3851 non-null   object 
 4   distance             3851 non-null   object 
 5   id                   3851 non-null   object 
 6   phone                3851 non-null   object 
 7   place_name           3851 non-null   object 
 8   place_url            3851 non-null   object 
 9   road_address_name    3851 non-null   object 
 10  x                    3851 non-null   object 
 11  y                    3851 non-null   object 
 12  평점                   3851 non-null   float64
dtypes: float64(1), object(12)
memory usage: 391.2+ KB


### 공통컬럼만

In [56]:
df_final = res_info_df.iloc[:,[0,3,7,8,10,11,12]]
df_final.columns = ['도로명주소', '카테고리', '상호명', '위도(y)', '경도(x)', '사이트', '별점']
df_final

,도로명주소,카테고리,상호명,위도(y),경도(x),사이트,별점
0,서울 양천구 신월동 110-29,"음식점 > 술집 > 호프,요리주점",돌솥바베큐,http://place.map.kakao.com/16847780,126.833882712082,37.5316406351089,5.00
1,서울 양천구 목동 601-2,음식점 > 한식,화이팅,http://place.map.kakao.com/16052371,126.86828451767764,37.54864378487966,5.00
2,서울 양천구 신정동 1294,음식점 > 분식,선가네분식,http://place.map.kakao.com/20540817,126.844478845194,37.5088957497724,5.00
3,서울 양천구 신월동 62-3,"음식점 > 술집 > 호프,요리주점",칼포니호프,http://place.map.kakao.com/365036181,126.832899709504,37.538667947389,5.00
4,서울 양천구 신월동 85-6,"음식점 > 술집 > 호프,요리주점",호프광장,http://place.map.kakao.com/16933981,126.833548267272,37.5375507231889,5.00
...,...,...,...,...,...,...,...
3846,서울 종로구 명륜3가 1-112,음식점 > 분식,김밥친구,http://place.map.kakao.com/20946662,126.995226500559,37.5885237796056,4.48
3847,서울 종로구 창신동 464-10,"음식점 > 한식 > 육류,고기 > 오리",시골집,http://place.map.kakao.com/11593474,127.011346923975,37.5714476946019,4.48
3848,서울 종로구 청진동 146,음식점 > 술집 > 일본식주점,카오리,http://place.map.kakao.com/89316784,126.979918914327,37.5708428652181,4.48
3849,서울 종로구 종로6가 238-2,음식점 > 한식 > 감자탕,방아다리감자국,http://place.map.kakao.com/8313401,127.004577790324,37.5727672079731,4.48


In [57]:
df_final['카테고리'].value_counts()

음식점 > 카페                          552
음식점 > 한식                          285
음식점 > 한식 > 육류,고기                  217
음식점 > 술집 > 호프,요리주점                194
음식점 > 카페 > 커피전문점                  149
                                 ... 
음식점 > 퓨전요리 > 퓨전한식 > 본죽&비빔밥cafe      1
음식점 > 카페 > 생과일전문점                   1
음식점 > 치킨 > 바비큐보스                    1
음식점 > 일식 > 타코비                      1
음식점 > 한식 > 순대 > 신의주찹쌀순대             1
Name: 카테고리, Length: 242, dtype: int64

### 최종 데이터프레임 내보내기

In [58]:
df_final.to_excel('식단품질정보_전처리완료.xlsx')